# Amperon Data Engineering Take Home Assignment

In [ ]:
!pip install ipython-sql
!pip install sqlalchemy
!pip install psycopg2
!pip install pandas
!pip install matplotlib

In [ ]:
import os
USERNAME = os.environ['PGUSER']
PASSWORD = os.environ['PGPASSWORD']
HOST = os.environ['PGHOST']
PORT = os.environ['PGPORT']
DATABASE = os.environ['PGDATABASE']


In [ ]:
%load_ext sql
%sql postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}

In [ ]:
%%sql
SELECT *
FROM tomorrow_io_timeline_data

# What's the latest temperature for each geolocation? What's the latest wind speed?


In [ ]:
%%sql
SELECT
    t.start_time,
    t.latitude,
    t.longitude,
    t.temperature_c,
    t.wind_speed
FROM
    tomorrow_io_timeline_data t
JOIN
    (
        SELECT
            latitude,
            longitude,
            MAX(start_time) AS latest_time
        FROM
            tomorrow_io_timeline_data
        GROUP BY
            latitude, longitude
    ) subquery
ON
    t.latitude = subquery.latitude
    AND t.longitude = subquery.longitude
    AND t.start_time = subquery.latest_time;

# Show an hourly time series of temperature from a day ago to 5 days in the future for location: (25.86, -97.42)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

weather_data = pd.read_sql('SELECT * FROM tomorrow_io_timeline_data', engine)

weather_data

In [ ]:
import matplotlib.pyplot as plt
latitude = 25.86
longitude = -97.42	

query = f"""
SELECT 
    start_time, 
    temperature_c 
FROM 
    tomorrow_io_timeline_data 
WHERE 
    latitude = {latitude}
    AND longitude = {longitude}
ORDER BY 
    start_time;
"""

df = pd.read_sql(query, engine)

plt.figure(figsize=(12, 6))
plt.plot(df['start_time'], df['temperature_c'], marker='o', linestyle='-')
plt.title(f'Hourly Temperature at location ({latitude}, {longitude})')
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()